<a href="https://colab.research.google.com/github/MatheusHenriqueDaSilva/data_science/blob/main/Exercicio_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split


In [83]:
# Data understanding
DATA = pd.read_csv('/content/classificacao_2.csv')
DATA.head()

,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,target
0,-0.79138,-1.45877,-0.61498,6.39546,-0.19283,-4.92609,3.72728,2.36581,0.60241,2.85597,2.52876,2.09155,0
1,0.79527,-0.14828,0.49140,0.10649,4.32203,-1.55355,-0.62966,-0.27509,4.06300,-1.11079,3.61077,1.11076,0
2,-2.30926,1.59887,1.23513,-4.49395,0.50298,4.10219,0.48356,-0.51920,-1.59507,1.75719,-1.71703,0.92375,1
3,1.97998,-2.71683,-0.73252,3.08358,3.52049,-0.61567,-0.06206,1.33451,0.75546,-1.56391,1.00917,1.66555,0
4,-1.22339,2.38761,-2.15763,-1.91316,1.31427,0.18067,-2.20922,1.43218,1.05498,0.18525,2.91384,-0.03258,1


In [84]:
def generate_metadata(dataframe):

    metadata2 = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata2=metadata2.sort_values(by='tipo')
    metadata2 = metadata2.reset_index(drop=True)

    return metadata2

metadados = generate_metadata(DATA)
metadados

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,target,int64,0,0.0,2
1,v_1,float64,0,0.0,2492
2,v_2,float64,0,0.0,2498
3,v_3,float64,0,0.0,2496
4,v_4,float64,0,0.0,2498
5,v_5,float64,0,0.0,2495
6,v_6,float64,0,0.0,2496
7,v_7,float64,0,0.0,2494
8,v_8,float64,0,0.0,2494
9,v_9,float64,0,0.0,2494


In [85]:
DATA.describe()

,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,target
count,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.00000
mean,-0.294870,-0.625044,-1.143620,1.654532,1.575157,-0.693946,-0.313119,1.001894,0.197870,0.024652,0.659996,0.673571,0.31040
std,2.016590,1.971944,2.568138,3.374940,3.250114,2.000317,2.055284,1.616507,1.910380,1.785112,1.771630,1.681005,0.46275
min,-6.888170,-7.622640,-11.218150,-13.460210,-8.306300,-7.320080,-7.281500,-4.799820,-5.703320,-7.561650,-6.355410,-5.615440,0.00000
25%,-1.636715,-2.014735,-2.758370,-0.326562,-0.724565,-2.015950,-1.668013,-0.075290,-1.099295,-1.183745,-0.469398,-0.410772,0.00000
50%,-0.366050,-0.659070,-1.206590,2.053625,1.428920,-0.778935,-0.241615,0.991420,0.137070,0.060500,0.704480,0.702765,0.00000
75%,1.051010,0.706873,0.463895,3.900132,3.676160,0.538633,1.086718,2.072715,1.493060,1.267752,1.746880,1.849287,1.00000
max,6.308010,6.264330,8.212290,11.906690,14.146850,7.161070,5.598460,6.704330,7.881510,5.408170,8.396750,6.038660,1.00000


In [91]:
# criando o melhor preditor de light GBM
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.naive_bayes import GaussianNB

x_treino_final = DATA.drop('target', axis=1)
y_treino_final = DATA['target']


# Definindo o modelo
model = GaussianNB()

# Definindo os parâmetros para o grid search
param_grid = {
    'var_smoothing': [1e-09]
}

# Definindo o objeto GridSearchCV
grid = GridSearchCV(model,
                    param_grid,
                    cv=KFold(n_splits = 10, shuffle=False), # padrão de shuffle é False
                    verbose=2,
                    scoring='roc_auc',  # 'accuracy' 'precision' 'recall' 'roc_auc' 'f1'
                    n_jobs=-1,
                    return_train_score = True)

# Treinando o modelo com o grid search
grid.fit(x_treino_final, y_treino_final)

# Exibindo os melhores parâmetros encontrados pelo grid search
print("Melhores Parâmetros: ", grid.best_params_)

# Exibindo a melhor pontuação (score) atingida pelo modelo com os melhores parâmetros
print("Melhor auc: ", grid.best_score_)




best_model = grid.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Melhores Parâmetros:  {'var_smoothing': 1e-09}
Melhor auc:  0.8576558604137453


In [80]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix,f1_score, classification_report


predict = best_model.predict(x_treino_final)

print(confusion_matrix(y_treino_final, predict))
print(f1_score(y_treino_final, predict))
print(classification_report(y_treino_final, predict))

[[493 105]
 [123 529]]
0.8227060653188181
              precision    recall  f1-score   support

           0       0.80      0.82      0.81       598
           1       0.83      0.81      0.82       652

    accuracy                           0.82      1250
   macro avg       0.82      0.82      0.82      1250
weighted avg       0.82      0.82      0.82      1250



In [92]:
estudo = pd.DataFrame(grid.cv_results_ )
estudo['mean_train_score']

0    0.859988
Name: mean_train_score, dtype: float64

In [93]:
estudo['mean_test_score']

0    0.857656
Name: mean_test_score, dtype: float64